In [1]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs

import requests
from requests.sessions import Session

import numpy as np
from scipy import stats
import datetime
import time
import statistics as st
import concurrent
from threading import Thread,local
import asyncio
import aiohttp
import tqdm
import nest_asyncio
import re

nest_asyncio.apply()

In [2]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

class DataError(ValueError): pass
class PMUError(Exception): pass

class Partants_scrapper():
    def __init__(self, course, training=True):
        self.course = course
        self.courseId = course["id"]
        self.date = course["date"].strftime("%Y-%m-%d")
        self.idHippo = course["idHippo"]
        self.numCourse = course["numCourse"]
        self.numReunion = course["numReunion"]
        self.heure = course["heureCourse"]
        self.has_tracking = course["hasTracking"]
        self.r_tab_partant = None
        self.r_tab_arrivee = None
        self.r_partant_pmu = None
        
        self.distance = course["distance"]
        self.categorie = course["categorie"].split(" ")[1]
        
        self.training = training
        self.classement = None
        self.gains = None
        
        try:
            loop = asyncio.get_event_loop()
            programme = loop.run_until_complete(asyncio.gather(self._info_tableau_partant()))

            df = pd.DataFrame(programme[0])
            df["rid"] = df["id"]
            df.drop('id', inplace=True, axis=1)
            if 'tauxReclamation' in list(df):
                df.drop('tauxReclamation', inplace=True, axis=1)
                
            if 'commentaireApresCourse_texte' in list(df):
                df.drop('commentaireApresCourse_texte', inplace=True, axis=1)   
            if 'commentaireApresCourse_source' in list(df):
                df.drop('commentaireApresCourse_source', inplace=True, axis=1)
            if 'handicapPoids' in list(df):
                df.drop('handicapPoids', inplace=True, axis=1) 
            if 'nomPereMere' in list(df):
                df.drop('nomPereMere', inplace=True, axis=1)
            self.info_partants = df.to_dict('records')
        except Exception as e:
            self.info_partants = None
            
    def _reduc_to_sec(self,reduc):
        minutes = int(reduc[0])
        secondes = int(reduc[1:3])
        dixieme = int(reduc[3])
        return (60  * minutes + secondes + dixieme/10)*1000
        
        
    async def _request_tableau_partants(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/partants/tableau", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        self.r_tab_partant = rows, headers_table
    
    async def _request_tableau_arrive(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/resultats/arrivee-definitive", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        classement = {row.select("td")[1].text : row.select("td")[0].find("span", {"class": "bold"}).text for row in rows}
        gains = {row.select("td")[1].text : row.find("div", {"class": "gains"}).text.replace(" ", "").replace("€", "") for row in rows}
        self.classement = classement
        self.gains = gains
        self.r_tab_arrivee = rows,classement
        
    async def _request_tableau_tracking(self, session):
        async with session.get(f"https://www.letrot.com/stats/fiche-course/{self.date}/{self.idHippo}/{self.numCourse}/tracking", headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        tracking = []
        for r in rows:
            cheval = {}
            cheval["num_tracking"] = int(r.find("td", {"title": "Numéro"}).text)
            cheval["distPremArriv"] = int(r.find("td", {"title": "Distance par rapport au 1er à l'arrivée en mètres"}).span.text)
            cheval["reduc2000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 2 000 m"}).span.text)
            cheval["class2000m"] = int(r.find("td", {"title": "Classement aux 2 000 m"}).span.text)
            
            cheval["reduc1500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 1 500 m"}).span.text)
            cheval["class1500m"] = int(r.find("td", {"title": "Classement aux 1 500 m"}).span.text)
            
            cheval["reduc1000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 1 000 m"}).span.text)
            cheval["class1000m"] = int(r.find("td", {"title": "Classement aux 1 000 m"}).span.text)   
            
            cheval["reduc500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km aux 500 m"}).span.text)
            cheval["class500m"] = int(r.find("td", {"title": "Classement aux 500 m"}).span.text)
            
            cheval["reducLast1000m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km derniers 1000 m"}).span.text)
            cheval["reducLast500m"] = self._reduc_to_sec(r.find("td", {"title": "Réduction km derniers 500 m"}).span.text)
            
            tracking.append(cheval)
            
        tracking.sort(key=lambda x: x["num_tracking"])
        
        self.tracking = tracking
    
    async def _request_partant_pmu(self, session):
        date_pmu = "".join(self.date.split("-")[::-1]) 
        async with session.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/R{self.numReunion}/C{self.numCourse}/participants", headers=headers) as response:
             participants_pmu = await response.json()
        try:
            pmu_jsoned = participants_pmu["participants"]
            participants = pd.json_normalize(pmu_jsoned, sep="_").to_dict(orient="records")
            participants_with_id = [dict(item, **{"id": self.courseId, "numReunion": self.numReunion}) for item in participants]  
            self.r_partant_pmu = participants_with_id
        except:
            raise PMUError("Erreur API PMU")
            
        
        
    async def _info_tableau_partant(self):
        chevaux = []
        
        tasks = []
        async with aiohttp.ClientSession() as session:
            tasks.append(asyncio.ensure_future(self._request_tableau_partants(session)))
            tasks.append(asyncio.ensure_future(self._request_tableau_arrive(session)))
            tasks.append(asyncio.ensure_future(self._request_partant_pmu(session)))
#             tasks.append(asyncio.ensure_future(self.get_info_couple(session)))
#             tasks.append(asyncio.ensure_future(self.get_info_cheval_hippo(session)))
            if self.has_tracking:
                tasks.append(asyncio.ensure_future(self._request_tableau_tracking(session)))
            res = await asyncio.gather(*tasks, return_exceptions=True)
            
        for r in res:
            if type(r) == PMUError:
                raise PMUError("Erreur PMU")
       
        try:
            tableau_partants, headers_table = self.r_tab_partant
            tableau_arrivee,classement = self.r_tab_arrivee
            tableau_pmu = self.r_partant_pmu
        except:
            raise DataError("Erreur donnée")
        
        chevaux.extend(tableau_pmu)
        
        for i,row in enumerate(tableau_partants):
                num = row.select("td")[0].find("span", {"class": "bold"}).text
                col = row.select("td")
                cheval = {}
                cheval["num"] = num
                cheval["nom"] = col[1].text
                
                if self.has_tracking and int(num) == self.tracking[i]["num_tracking"] and len(self.tracking) == len(tableau_partants):
                    cheval.update(self.tracking[i])
                
                cheval["numCoursePMU"] = f"R{self.numReunion}C{self.numCourse}"

                if self.training:
                    if num == "NP":
                        cheval["classement"] = "NP"
                    else:
                        cheval["classement"] = classement[num]
                      
                if num != "NP":
                    cheval["gainCourse"] = int(self.gains[num])
                else:
                    cheval["gainCourse"] = 0
                    
                cheval["id"] = self.courseId
                cheval["date"] = self.date
                cheval["url"] = col[1].find("a").get("href")

                cheval["heureCourse"] = self.heure
                cheval["fer"] = int(col[3].text) if col[3].text else 0
                cheval["firstTimeFer"] = 1 if col[3].find("div", {"class", "fer-first-time"}) else 0
                cheval["sex"] = 0 if col[4].text == "M" else 1
                cheval["age"] = int(col[5].text)
                cheval["dist"] = int(col[6].text.replace(" ", "").replace("\n", ""))
                cheval["driver"] = col[7].find("a").get("href")
                cheval["trainer"] = col[8].find("a").get("href")
                
                cheval["driver_id"] = cheval["driver"].split("/")[-3]
                cheval["trainer_id"] = cheval["trainer"].split("/")[-3]

                if "Avis" in headers_table[9].text:
                    cheval["avisTrainer"] = int(col[9].get("data-order"))
                    avis = col.pop(9)
                    col.insert(-1, avis)
                else:
                    cheval["avisTrainer"] = 2
                
                music = list(filter(lambda x: "a" in x, col[9].text.replace("D", "0").replace("Ret", "0").replace("T", "0").split()))

                try:
                    cheval["recordAbs"] = list(map(int, col[10].text.replace(col[10].span.text, "").replace("\'", '"').split('"')))
                    cheval["recordAbs"] = cheval["recordAbs"][0] * 10 * 60 + cheval["recordAbs"][1] * 10 + cheval["recordAbs"][2]
                except:
                    cheval["recordAbs"] = None

                cheval["gain"] = int(col[11].find("div", class_="gains").text.replace(" ", "")[:-1])
                
                chevaux[i].update(cheval)
        return chevaux
    
    
    async def get_tracking(self,session, url):
        async with session.get(url.replace("dernieres-performances", "tracking"), headers=headers) as response:
             r = await response.text()
        soup = bs(r, "html.parser")
        headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
        table = soup.find("table", {"id": "result_table"}).find("tbody")
        rows = table.find_all("tr")
        
        info_tracking = {}
        
        distance_au_premier_arrivee = []
        accélération_500m = []
        gain_classement_500m = []
        for row in rows:
            dist_prem = int(row.find_all("td")[2].span.text)
            if dist_prem < 9999:
                distance_au_premier_arrivee.append(dist_prem)
            
            pre_fin = int(row.find_all("td")[17].span.text)
            fin = int(row.find_all("td")[18].span.text)
            if pre_fin < 2000 and fin < 2000:                                            
                accélération_500m.append(pre_fin - fin)
                

            try:
                class_500m = int(row.find_all("td")[16].span.text)
                class_final = int(row.find_all("td")[1].find("span", {"class": "bold"}).text)
                if class_500m -  class_final < 10:
                    gain_classement_500m.append(class_500m -  class_final)
            except:
                gain_classement_500m.append(0)
            
        info_tracking["mean_dist_arrivee"] = np.mean(distance_au_premier_arrivee) if len(distance_au_premier_arrivee) > 0 else np.nan
        info_tracking["acceleration_500m"] = np.mean(accélération_500m) if len(accélération_500m) > 0 else np.nan
        info_tracking["gain_classement_fin"] = np.mean(gain_classement_500m) if len(gain_classement_500m) > 0 else np.nan
        return info_tracking

In [643]:
courses_df = pd.read_csv("../data/courses1922.csv", index_col=0)
courses_df["distance"] = courses_df["distance"].str.replace(" ", "").astype(int)
courses_df["date"] = courses_df["date"] + " " + courses_df["heureCourse"]
courses_df["date"] = pd.to_datetime(courses_df["date"])
courses_df.head()

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2019-01-01 14:02:00,2019010175001,1,VINCENNES,7500,14:02,Attelé,1,PRIX DE PITHIVIERS,47 000,2700,Course B,NaN,"Pour pouliches de 4 ans (F), n'ayant pas gagné...",True,False,16,5 - 11 - 3 - 8 - 4,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2019-01-01 14:36:00,2019010175002,1,VINCENNES,7500,14:36,Attelé,2,PRIX PIERRE RAFFRE,35 000,2850,Course D,NaN,Course D<br />Pour chevaux entiers et hongres ...,False,False,16,4 - 5 - 9 - 3 - 7,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2019-01-01 15:15:00,2019010175003,1,VINCENNES,7500,15:15,Attelé,3,PRIX DU CROISE-LAROCHE,54 000,2850,Course C,NaN,"Pour 8 et 9 ans (B et A), n'ayant pas gagné 23...",True,False,16,11 - 2 - 13 - 15 - 14,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2019-01-01 15:52:00,2019010175004,1,VINCENNES,7500,15:52,Attelé,4,PRIX DE CHARLEVILLE,39 000,2100,Course D,NaN,Course Européenne<br />Départ à l'autostart<br...,False,False,16,6 - 1 - 11 - 9 - 12,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2019-01-01 16:25:00,2019010175005,1,VINCENNES,7500,16:25,Attelé,5,PRIX D'ANGOULEME,63 000,2100,Course A,NaN,Course Européenne<br />Départ à l'autostart<br...,True,False,16,6 - 1 - 12 - 4 - 11,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [622]:
testcourses = pd.read_sql("SELECT * FROM courses", con=engine)
testcourses["date"] = pd.to_datetime(testcourses["date"])

In [624]:
testcourses

,id,date,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2022091782013,2022-09-17 15:00:00,6,BEAUMONT DE LOMAGNE,8201,15:00,Attelé,3,PRIX D'AVENCHES,16 000,2400,Course F,None,Départ à l'autostart<br />Pour juments de 5 an...,0,0,16,1 - 2 - 12 - 8 - 11,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2022091782015,2022-09-17 16:00:00,6,BEAUMONT DE LOMAGNE,8201,16:00,Attelé,5,PRIX MEADOWLAND (Gr A),13 000,2400,Course F,None,"Départ à l'autostart<br />Pour 4 ans, n'ayant ...",0,0,16,12 - 6 - 11 - 8 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2022091782016,2022-09-17 16:30:00,6,BEAUMONT DE LOMAGNE,8201,16:30,Attelé,6,PX DE LA COMMUNAUTE DE COMMUNES TARN&GAR,13 000,2400,Course F,None,"Départ à l'autostart<br />Pour 4 ans, n'ayant ...",0,0,16,8 - 11 - 16 - 1 - 10,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2022091782017,2022-09-17 17:00:00,6,BEAUMONT DE LOMAGNE,8201,17:00,Attelé,7,PRIX DE WOLVEGA,20 000,2550,Course F,None,"Pour 7 et 8 ans, n'ayant pas gagné 124.000. - ...",0,0,16,3 - 10 - 6 - 17 - 9,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2022091782018,2022-09-17 17:30:00,6,BEAUMONT DE LOMAGNE,8201,17:30,Attelé,8,PRIX DE GOTEBORG ABY,21 000,2550,Course F,None,"Pour 6 à 9 ans inclus, n'ayant pas gagné 149.0...",0,0,16,7 - 8 - 4 - 9 - 1,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,2021032014076,2021-03-20 13:42:00,3,LISIEUX,1407,13:42,Attelé,6,PRIX VILLE DE LISIEUX,15 000,2725,Course F,None,"Pour poulains entiers et hongres de 4 ans, n'a...",0,1,16,4 - 7 - 13 - 6 - 8,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
12679,2021032151027,2021-03-21 18:30:00,4,REIMS,5102,18:30,Attelé,7,PRIX DES PAQUERETTES,16 000,2550,Course F,None,"Pour 3 ans, n'ayant pas gagné 5.000.<br />",0,0,16,13 - 6 - 5 - 10 - 9,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
12680,2021032214002,2021-03-22 15:36:00,4,CAEN,1400,15:36,Attelé,2,PRIX DE DEMONVILLE,18 000,2450,Course E,None,"Pour poulains entiers et hongres de 3 ans, n'a...",0,0,16,2 - 7 - 9 - 8 - 10,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."
12681,2021032214004,2021-03-22 16:36:00,4,CAEN,1400,16:36,Attelé,4,PRIX DE BENERVILLE,18 000,2200,Course E,None,Départ à l'autostart<br />Pour poulains entier...,0,1,16,3 - 7 - 5 - 6 - 1,https://www.letrot.com/stats/fiche-course/2021...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [651]:
info_2 = []
t = time.time()
counter = 0

def gen_rows(df):
    for row in df.itertuples(index=False):
        yield row._asdict()
# ids = pd.read_csv("data/final_data_2122.csv").id.unique()
# c_filtered = c.loc[~c.id.isin(ids)]

for row in gen_rows(courses_df[~courses_df["id"].astype(str).isin(courses_saved.id.unique())]):
    res = Partants_scrapper(row)
    if isinstance(res.info_partants, list):
        try:
            with Session(engine) as session:
                partants = [Partant(**i) for i in res.info_partants]
                courses = [Course(**row, partants=partants)]
                session.add_all(courses)
                session.commit()
        except Exception as e:
            print(e)
            continue
#         info_2.extend(res.info_partants)
#         pd.DataFrame(res.info_partants).to_csv("data/final_data_2122.csv",mode="a", header=not os.path.isfile("data/final_data_2122.csv") ,index=False)
tps = time.time() - t
print(f"Fini en {int(tps//60)}min {int(tps%60)}s")

'handicapPoids' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an inv

'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_s

'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
'commentaireApresCourse_source' is an invalid keyword argument for Partant
Fini en 28min 32s


In [628]:
df = pd.DataFrame(info_2)
df["gainCourse"]

0         0
1         0
2         0
3         0
4       400
5      1600
6         0
7       200
8         0
9         0
10     1000
11        0
12        0
13     2800
14     5000
15     9000
16        0
17        0
18     1150
19      230
20        0
21        0
22        0
23        0
24    10350
25     1840
26        0
27      460
28        0
29        0
30     3220
31        0
32     5750
33        0
34        0
35      200
36     2800
37     1000
38     9000
39        0
40     5000
41        0
42      400
43     1600
44        0
Name: gainCourse, dtype: int64

In [581]:
list(test)

['id',
 'nom',
 'numPmu',
 'age',
 'sexe',
 'race',
 'statut',
 'placeCorde',
 'oeilleres',
 'proprietaire',
 'entraineur',
 'driver',
 'driverChange',
 'indicateurInedit',
 'musique',
 'nombreCourses',
 'nombreVictoires',
 'nombrePlaces',
 'nomPere',
 'nomMere',
 'ordreArrivee',
 'jumentPleine',
 'engagement',
 'supplement',
 'handicapDistance',
 'poidsConditionMonteChange',
 'tempsObtenu',
 'reductionKilometrique',
 'allure',
 'robe_code',
 'robe_libelleCourt',
 'robe_libelleLong',
 'dernierRapportDirect_typePari',
 'dernierRapportDirect_rapport',
 'dernierRapportDirect_typeRapport',
 'dernierRapportDirect_indicateurTendance',
 'dernierRapportDirect_nombreIndicateurTendance',
 'dernierRapportDirect_dateRapport',
 'dernierRapportDirect_permutation',
 'dernierRapportDirect_favoris',
 'dernierRapportDirect_numPmu1',
 'dernierRapportDirect_grossePrise',
 'deferre',
 'dernierRapportReference_typePari',
 'dernierRapportReference_rapport',
 'dernierRapportReference_typeRapport',
 'dernierRa

In [174]:
test = pd.read_sql("SELECT * FROM partants WHERE date BETWEEN '2021-01-01' AND '2021-12-22' ORDER BY date", con=engine)
test["date"] = pd.to_datetime(test["date"])
test.sort_values(["date","rid","numPmu"], inplace=True)
test.reset_index(drop=True, inplace=True)
# test["accel500m"] = (test["reducLast1000m"] - test["reducLast500m"])
test[["rid","nom","classement","gainCourse"]].tail()

,rid,nom,classement,gainCourse
54038,2021122275008,IMPRESSION,7,380
54039,2021122275008,ISABELLA FERM,3,5320
54040,2021122275008,ILE AUX MOINES,2,9500
54041,2021122275008,IRONIE D'HERIPRE,0,0
54042,2021122275008,INES DE BLARY,1,17100


In [175]:
test["driver_id"]

0        Y2N3YwEFYA
1        Y2RbZwoAfw
2        Z2t8YQAFdw
3        YGJaZQMFYQ
4        Y2J6YwUDdw
            ...    
54038    Z2JbZgIAeg
54039    Z2Z6ZwECfA
54040    YGJaZwQFZA
54041    bGN6YAUIeQ
54042    ZGtaZAQCdA
Name: driver_id, Length: 54043, dtype: object

In [176]:
test["lastRace"] = test.groupby("nom")["date"].transform(lambda x: x - x.shift(1))
test["lastRace_driver"] = test.groupby("driver_id")["date"].transform(lambda x: x - x.shift(1))

In [177]:
test["classLastRace_horse"] = test.groupby("nom")["classement"].transform(lambda x: x.shift(1))
test["classLastRace_driver"] = test.groupby("driver_id")["classement"].transform(lambda x: x.shift(1))

In [178]:
test[["date","nom","lastRace","classement","classLastRace"]].tail(20)

,date,nom,lastRace,classement,classLastRace
54023,2021-12-22,FOLIE DE HOUELLE,10 days,9,0
54024,2021-12-22,FERIA DU METZ,16 days,DA,2
54025,2021-12-22,FLORA VENESI,16 days,7,DA
54026,2021-12-22,FABULEUSE FLIGNY,16 days,0,6
54027,2021-12-22,FEE DE LUNOJO,23 days,3,3
54028,2021-12-22,ISHTAR FONT,78 days,0,0
54029,2021-12-22,IRIS DES CHAMPS,10 days,0,DA
54030,2021-12-22,INFANTE D'ERABLE,78 days,8,4
54031,2021-12-22,INDEPENDANTE,16 days,9,DA
54032,2021-12-22,ILLUSION JIPAD,7 days,6,1


In [179]:
groupnom = test.groupby("nom")
test["weightDist"] = np.max((test["gainCourse"].to_numpy(), np.ones(len(test["gainCourse"]))), axis=0)
test["weightDistSum"] = groupnom["weightDist"].transform(lambda x: x.shift(1).rolling(20, min_periods=1).sum())
test["distProd"] = test["weightDist"] * test["dist"]
test["distProdSum"] = groupnom["distProd"].transform(lambda x: x.shift(1).rolling(20, min_periods=1).sum())
test["preferedDist"] =  test["distProdSum"]/test["weightDistSum"] 
test["preferedDist"]

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
            ...     
54038    2250.772852
54039    2200.144732
54040    2252.807924
54041    2437.500000
54042    2354.990720
Name: preferedDist, Length: 54043, dtype: float64

In [180]:
groupdriver = test.groupby("driver_id")
test["weightDistSum_driver"] = groupdriver["weightDist"].transform(lambda x: x.shift(1).rolling(50, min_periods=1).sum())
test["distProdSum_driver"] = groupdriver["distProd"].transform(lambda x: x.shift(1).rolling(50, min_periods=1).sum())
test["preferedDist_driver"] =  test["distProdSum_driver"]/test["weightDistSum_driver"] 
test["preferedDist_driver"]

0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
            ...     
54038    2539.471015
54039    2609.741765
54040    2732.218517
54041    2643.606049
54042    2426.216316
Name: preferedDist_driver, Length: 54043, dtype: float64

In [181]:
test["lastRaceDist"] = groupnom["dist"].transform(lambda x: x.shift(1))

In [182]:
test[["driver_id"]]

,driver_id
0,Y2N3YwEFYA
1,Y2RbZwoAfw
2,Z2t8YQAFdw
3,YGJaZQMFYQ
4,Y2J6YwUDdw
...,...
54038,Z2JbZgIAeg
54039,Z2Z6ZwECfA
54040,YGJaZwQFZA
54041,bGN6YAUIeQ


In [183]:
test.loc[test["driver_id"] == "YWR4bAEEfA"][["date","nom","driver","lastRace_driver","classement","classLastRace","gainCourse","dist","lastRaceDist","preferedDist_driver"]]

,date,nom,driver,lastRace_driver,classement,classLastRace,gainCourse,dist,lastRaceDist,preferedDist_driver
1134,2021-01-11,HADOL DU CHATELET,https://www.letrot.com/stats/fiche-homme/david...,NaT,DA,NaN,0,2850,NaN,NaN
1169,2021-01-11,IVAKIR DU VARLET,https://www.letrot.com/stats/fiche-homme/david...,0 days,9,NaN,0,2850,NaN,2850.000000
2417,2021-01-22,HILTON BERRY,https://www.letrot.com/stats/fiche-homme/david...,11 days,DA,NaN,0,2850,NaN,2850.000000
4128,2021-02-06,EMILE DU VAR,https://www.letrot.com/stats/fiche-homme/david...,15 days,6,NaN,400,2850,NaN,2850.000000
6502,2021-02-27,HAPPY DU LION,https://www.letrot.com/stats/fiche-homme/david...,21 days,9,NaN,0,2400,NaN,2850.000000
...,...,...,...,...,...,...,...,...,...,...
51020,2021-12-05,IP OP DE LA ROQUE,https://www.letrot.com/stats/fiche-homme/david...,0 days,6,7,260,2850,2900.0,2640.784726
51253,2021-12-06,ITALIE,https://www.letrot.com/stats/fiche-homme/david...,1 days,5,2,650,2825,2825.0,2642.376053
51266,2021-12-06,HEROINE DE PUCH,https://www.letrot.com/stats/fiche-homme/david...,0 days,DA,7,0,2825,2400.0,2645.790846
53844,2021-12-21,HOOKER DU PECH,https://www.letrot.com/stats/fiche-homme/david...,15 days,DA,3,0,2400,2725.0,2645.794446


In [84]:
test["distPremMean"] = test.groupby("nom")["distPremArriv"].transform(lambda x: x.rolling(5, min_periods=1).mean(skipna=True).fillna(method='bfill'))

In [145]:
test[["nom","date","distPremMean"]]

,nom,date,distPremMean
0,HORATIUS D'ELA,2021-01-01,13.0
1,HORSY ROYAL,2021-01-01,9999.0
2,HERAKLION DESBOIS,2021-01-01,9999.0
3,HACKER LOVER,2021-01-01,5.0
4,HIDELTO DE HOUELLE,2021-01-01,11.0
...,...,...,...
48857,HAKIRA SCOTT,2021-11-21,NaN
48858,HASKIDO THISACLE,2021-11-21,NaN
48859,HERMIONE DU PONT,2021-11-21,NaN
48860,HASTIG VASTERBO,2021-11-21,NaN


In [85]:
test["musique"]

0            5a5aDa4a1aDa1a1a7a
1                  1a1a1aDaDa1a
2          0aDa5a3aDa7a3a4a3a9a
3        6a7a6aDmDm1m1a(19)2m7a
4          Da3a3a4a3a3a3a3aDa4a
                  ...          
48857      6aDa6aDa0a0aDa6a3a4a
48858      7a0a3a7aDaDa3a2a4a3a
48859    5a0aDa7a2aDaDa(20)7a2a
48860    Da1aDa1aDa7aDaDa(20)Da
48861      4a0a8a6a0a2a6a1a3a6a
Name: musique, Length: 54043, dtype: object

In [139]:
def parse_musique(mus):
    res = re.findall(r'([0-9D])a',mus)
    res = list(map(lambda x: int(x.replace("0", "10").replace("D", "10")), res))
    if len(res) >0:
        return res
    else:
        return [0]

In [142]:
test["meanMusic"] = test["musique"].apply(lambda x: np.mean(parse_musique(x)))
test["medianMusic"] = test["musique"].apply(lambda x: np.median(parse_musique(x)))
test["minMusic"] = test["musique"].apply(lambda x: np.min(parse_musique(x)))

In [143]:
test[["meanMusic","medianMusic","minMusic"]]

,meanMusic,medianMusic,minMusic
0,4.888889,5.0,1
1,4.000000,1.0,1
2,6.400000,6.0,3
3,5.400000,6.0,1
4,4.600000,3.0,3
...,...,...,...
48857,7.500000,8.0,3
48858,5.900000,5.5,2
48859,7.000000,7.0,2
48860,7.666667,10.0,1


In [121]:
parse_musique("4a0a8a6a0a2a6a1a3a6a")

[4, 10, 8, 6, 10, 2, 6, 1, 3, 6]


(5.6, 6.0, 1)

In [138]:
test["musique"].apply(parse_musique).to_numpy()

array([list([4.888888888888889, 5.0, 1]), list([4.0, 1.0, 1]),
       list([6.4, 6.0, 3]), ..., list([7.0, 7.0, 2]),
       list([7.666666666666667, 10.0, 1]), list([5.6, 6.0, 1])],
      dtype=object)

In [596]:
test["distPremArriv"].replace(9999.0, np.nan, inplace=True)

In [652]:
courses_saved = pd.read_sql("SELECT * FROM courses", con=engine)

In [653]:
len(courses_df) - len(courses_saved)

2465

In [654]:
courses_df[~courses_df["id"].astype(str).isin(courses_saved.id.unique())]

,date,id,numReunion,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2019-01-01 14:02:00,2019010175001,1,VINCENNES,7500,14:02,Attelé,1,PRIX DE PITHIVIERS,47 000,2700,Course B,NaN,"Pour pouliches de 4 ans (F), n'ayant pas gagné...",True,False,16,5 - 11 - 3 - 8 - 4,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
9,2019-01-02 15:12:00,2019010275007,2,VINCENNES,7500,15:12,Attelé,7,PRIX D'ACIGNE,29 000,2175,Course D,NaN,"Pour pouliches de 3 ans (G), n'ayant pas gagné...",True,False,16,11 - 14 - 8 - 6 - 13,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
39,2019-01-06 15:15:00,2019010675004,1,VINCENNES,7500,15:15,Attelé,4,PRIX LE PARISIEN (PRIX DE LILLE),75 000,2100,Course A,NaN,Course Européenne<br />Départ à l'autostart<br...,True,False,16,8 - 5 - 12 - 1 - 6,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
40,2019-01-06 16:31:00,2019010675006,1,VINCENNES,7500,16:31,Attelé,6,PRIX DE VIC SUR CERE,63 000,2850,Course A,NaN,Course Européenne<br />Pour chevaux entiers et...,True,False,16,11 - 7 - 12 - 3 - 4,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
43,2019-01-07 13:45:00,2019010775001,1,VINCENNES,7500,13:45,Attelé,1,PRIX DE L'AVEYRON,54 000,2850,Course C,NaN,"Pour juments de 7, 8 et 9 ans (C, B et A), n'a...",True,False,16,3 - 4 - 2 - 1 - 11,https://www.letrot.com/stats/fiche-course/2019...,"<a href=""https://www.letrot.com/fr/replay-cour..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18330,2022-12-22 12:52:00,2022122253073,4,MESLAY-DU-MAINE,5307,12:52,Attelé,3,PRIX DES CADEAUX,20 000,2600,Course F,NaN,Départ à l'autostart<br />Pour juments de 5 an...,False,False,16,5 - 8 - 2 - 16 - 3,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
18331,2022-12-22 13:25:00,2022122253074,4,MESLAY-DU-MAINE,5307,13:25,Attelé,4,PRIX DE LA NEIGE,20 000,2600,Course F,NaN,Départ à l'autostart<br />Pour chevaux entiers...,False,False,16,8 - 15 - 7 - 4 - 6,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
18332,2022-12-22 14:46:00,2022122253076,4,MESLAY-DU-MAINE,5307,14:46,Attelé,6,PRIX DES GUIRLANDES,17 000,2875,Course F,NaN,"Pour 3 ans, n'ayant pas gagné 9.400.<br />",False,False,16,14 - 1 - 12 - 7 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
18333,2022-12-22 15:22:00,2022122253077,4,MESLAY-DU-MAINE,5307,15:22,Attelé,7,PRIX DE NOEL,22 000,2875,Course E,NaN,"Pour 6 ans, n'ayant pas gagné 114.000. - Recul...",False,False,16,18 - 5 - 12 - 4 - 14,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."


# SQL

In [3]:
from sqlalchemy import create_engine,ForeignKey
from sqlalchemy.orm import declarative_base,relationship
from sqlalchemy import Column, Integer, String, Boolean, BIGINT, TEXT, INT, FLOAT, BOOLEAN
from sqlalchemy.dialects.sqlite import DATETIME
from sqlalchemy.orm import Session
Base = declarative_base()

engine = create_engine("sqlite:///test.sqlite", echo=False)

In [552]:
# df.to_sql('partants', con=engine)

In [630]:
class Course(Base):
    __tablename__ = "courses"

#     id = Column(Integer, primary_key=True)
    id = Column(TEXT, primary_key=True)
    date = Column(DATETIME)
    numReunion = Column(INT)
    hippodrome = Column(TEXT)
    idHippo = Column(INT)
    heureCourse = Column(TEXT)
    discipline = Column(TEXT)
    numCourse = Column(INT)
    prix = Column(TEXT)
    allocation = Column(TEXT)
    distance = Column(INT)
    categorie = Column(TEXT)
    typePiste = Column(TEXT)
    conditionsEngagement = Column(TEXT)
    hasTracking = Column(BOOLEAN)
    hasVideoHeat = Column(BOOLEAN)
    statut = Column(INT)
    classement = Column(TEXT)
    linkPrix = Column(TEXT)
    replay = Column(TEXT)
    
    partants = relationship("Partant", order_by=Partant.numPmu, back_populates="course")
    
    def __repr__(self):
        return "<Course(id='%s', date='%s', numReunion='%s', hippodrome='%s')>" % (
            self.id,
            self.date,
            self.numReunion,
            self.hippodrome,
        )

In [631]:
class Partant(Base):
    __tablename__ = "partants"

    id = Column(Integer, primary_key=True)
    
    rid = Column(INT)
    nom = Column(TEXT)
    numPmu = Column(INT)
    age = Column(INT)
    sexe = Column(TEXT)
    race = Column(TEXT)
    statut = Column(TEXT)
    placeCorde = Column(FLOAT)
    oeilleres = Column(TEXT)
    proprietaire = Column(TEXT)
    entraineur = Column(TEXT)
    driver = Column(TEXT)
    driverChange = Column(BOOLEAN)
    indicateurInedit = Column(BOOLEAN)
    musique = Column(TEXT)
    nombreCourses = Column(INT)
    nombreVictoires = Column(INT)
    nombrePlaces = Column(INT)
    nomPere = Column(TEXT)
    nomMere = Column(TEXT)
    ordreArrivee = Column(FLOAT)
    jumentPleine = Column(BOOLEAN)
    engagement = Column(BOOLEAN)
    supplement = Column(INT)
    handicapDistance = Column(INT)
    poidsConditionMonteChange = Column(BOOLEAN)
    tempsObtenu = Column(FLOAT)
    reductionKilometrique = Column(FLOAT)
    allure = Column(TEXT)
    robe_code = Column(TEXT)
    robe_libelleCourt = Column(TEXT)
    robe_libelleLong = Column(TEXT)
    dernierRapportDirect_typePari = Column(TEXT)
    dernierRapportDirect_rapport = Column(FLOAT)
    dernierRapportDirect_typeRapport = Column(TEXT)
    dernierRapportDirect_indicateurTendance = Column(TEXT)
    dernierRapportDirect_nombreIndicateurTendance = Column(FLOAT)
    dernierRapportDirect_dateRapport = Column(FLOAT)
    dernierRapportDirect_permutation = Column(FLOAT)
    dernierRapportDirect_favoris = Column(TEXT)
    dernierRapportDirect_numPmu1 = Column(FLOAT)
    dernierRapportDirect_grossePrise = Column(TEXT)
    deferre = Column(TEXT)
    dernierRapportReference_typePari = Column(TEXT)
    dernierRapportReference_rapport = Column(FLOAT)
    dernierRapportReference_typeRapport = Column(TEXT)
    dernierRapportReference_indicateurTendance = Column(TEXT)
    dernierRapportReference_nombreIndicateurTendance = Column(FLOAT)
    dernierRapportReference_dateRapport = Column(FLOAT)
    dernierRapportReference_permutation = Column(FLOAT)
    dernierRapportReference_favoris = Column(TEXT)
    dernierRapportReference_numPmu1 = Column(FLOAT)
    dernierRapportReference_grossePrise = Column(TEXT)
    incident = Column(TEXT)
    numReunion = Column(INT)
    num = Column(TEXT)
    numCoursePMU = Column(TEXT)
    classement = Column(TEXT)
    date = Column(TEXT)
    url = Column(TEXT)
    heureCourse = Column(TEXT)
    fer = Column(INT)
    firstTimeFer = Column(INT)
    sex = Column(INT)
    dist = Column(INT)
    trainer = Column(TEXT)
    driver_id = Column(TEXT)
    trainer_id = Column(TEXT)
    avisTrainer = Column(INT)
    recordAbs = Column(FLOAT)
    gain = Column(INT)
    rid = Column(INT)
    ecurie = Column(TEXT)
    urlCasaque = Column(TEXT)
    nombrePlacesSecond = Column(FLOAT)
    nombrePlacesTroisieme = Column(FLOAT)
    eleveur = Column(TEXT)
    avisEntraineur = Column(TEXT)
    gainCourse = Column(INT)
    gainsParticipant_gainsCarriere = Column(FLOAT)
    gainsParticipant_gainsVictoires = Column(FLOAT)
    gainsParticipant_gainsPlace = Column(FLOAT)
    gainsParticipant_gainsAnneeEnCours = Column(FLOAT)
    gainsParticipant_gainsAnneePrecedente = Column(FLOAT)
    poidsConditionMonte = Column(FLOAT)
    num_tracking = Column(FLOAT)
    distPremArriv = Column(FLOAT)
    reduc2000m = Column(FLOAT)
    class2000m = Column(FLOAT)
    reduc1500m = Column(FLOAT)
    class1500m = Column(FLOAT)
    reduc1000m = Column(FLOAT)
    class1000m = Column(FLOAT)
    reduc500m = Column(FLOAT)
    class500m = Column(FLOAT)
    reducLast1000m = Column(FLOAT)
    reducLast500m = Column(FLOAT)
    
    race_id = Column(Integer, ForeignKey("courses.id"))
    course = relationship("Course", back_populates="partants")

    def __repr__(self):
        return "<Partant(id='%s', numReunion='%s', classement='%s')>" % (
            self.course,
            self.numReunion,
            self.classement,
        )

In [632]:
Base.metadata.create_all(engine)

In [466]:
# with Session(engine) as session:
#     l_partant = df[df["rid"] == 2022091675001].to_dict("records")
# #     l_courses = courses_df[courses_df["id"] == 2022091675001].to_dict("records")
# #     print(l_courses)
#     partants = [Partant(**i) for i in l_partant]
#     # print(courses[courses["id"] == 2022091675001].to_dict("records")[0])
#     courses = [Course(**courses_df[courses_df["id"] == 2022091675001].to_dict("records")[0], partants=partants)]

#     session.add_all(courses)
#     session.commit()

2022-12-22 14:52:13,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-22 14:52:13,314 INFO sqlalchemy.engine.Engine INSERT INTO courses (id, date, "numReunion", hippodrome, "idHippo", "heureCourse", discipline, "numCourse", prix, allocation, distance, categorie, "typePiste", "conditionsEngagement", "hasTracking", "hasVideoHeat", statut, classement, "linkPrix", replay) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2022-12-22 14:52:13,315 INFO sqlalchemy.engine.Engine [cached since 316s ago] (2022091675001, '2022-09-16 18:58:00.000000', 1, 'VINCENNES', 7500, '18:58', 'Attelé', 1, 'PRIX LUCRETIA', '39 000', 2100, 'Course D', nan, "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />", 1, 0, 16, '7 - 4 - 10 - 15 - 3', 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive', '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: courses.id
[SQL: INSERT INTO courses (id, date, "numReunion", hippodrome, "idHippo", "heureCourse", discipline, "numCourse", prix, allocation, distance, categorie, "typePiste", "conditionsEngagement", "hasTracking", "hasVideoHeat", statut, classement, "linkPrix", replay) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (2022091675001, '2022-09-16 18:58:00.000000', 1, 'VINCENNES', 7500, '18:58', 'Attelé', 1, 'PRIX LUCRETIA', '39 000', 2100, 'Course D', nan, "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />", 1, 0, 16, '7 - 4 - 10 - 15 - 3', 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive', '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" ve ... (213 characters truncated) ... rm="translate(4.000000, 6.000000) rotate(9

In [327]:
for i in df[df["id"] == 2022091675001].to_dict("records"):
    print(i)

{'nom': 'INES PICARDE', 'numPmu': 1, 'age': 4, 'sexe': 'FEMELLES', 'race': 'TROTTEUR FRANCAIS', 'statut': 'PARTANT', 'placeCorde': 1.0, 'oeilleres': 'SANS_OEILLERES', 'proprietaire': 'Ecurie J-F. SENET', 'entraineur': 'J.F. SENET', 'deferre': 'DEFERRE_ANTERIEURS_POSTERIEURS', 'driver': 'https://www.letrot.com/stats/fiche-homme/jf-senet/bGV5ZwMDeQ/jockey/dernieres-courses', 'driverChange': False, 'indicateurInedit': False, 'musique': '0a6aDa0a7a8a1a8a(21)4a', 'nombreCourses': 33, 'nombreVictoires': 1, 'nombrePlaces': 15, 'nombrePlacesSecond': 4, 'nombrePlacesTroisieme': 2, 'nomPere': "ALADIN D'ECAJEUL", 'nomMere': 'SWEET RAINBOW', 'ordreArrivee': 13.0, 'jumentPleine': False, 'engagement': False, 'supplement': 0, 'handicapDistance': 2100, 'poidsConditionMonteChange': False, 'tempsObtenu': 154490.0, 'reductionKilometrique': 73600.0, 'urlCasaque': 'https://www.pmu.fr/back-assets/hippique/casaques/16092022/R1/C1/P1.png', 'eleveur': 'E.A.R.L. Ecurie des TROTTEURS PICARDS', 'allure': 'TROT', 

In [328]:
courses[courses["id"] == 2022091675001].to_dict("records")

[{'date': Timestamp('2022-09-16 18:58:00'),
  'id': 2022091675001,
  'numReunion': 1,
  'hippodrome': 'VINCENNES',
  'idHippo': 7500,
  'heureCourse': '18:58',
  'discipline': 'Attelé',
  'numCourse': 1,
  'prix': 'PRIX LUCRETIA',
  'allocation': '39 000',
  'distance': 2100,
  'categorie': 'Course D',
  'typePiste': nan,
  'conditionsEngagement': "Départ à l'autostart<br />Pour pouliches de 4 ans, n'ayant pas gagné 55.000.<br />",
  'hasTracking': True,
  'hasVideoHeat': False,
  'statut': 16,
  'classement': '7 - 4 - 10 - 15 - 3',
  'linkPrix': 'https://www.letrot.com/stats/fiche-course/2022-09-16/7500/1/resultats/arrivee-definitive',
  'replay': '<a href="https://www.letrot.com/fr/replay-courses/2022-09-16/7500/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><polygon id="triangle" fi

In [311]:
df[df["id"] == 2022091675001]

,nom,numPmu,age,sexe,race,statut,placeCorde,oeilleres,proprietaire,entraineur,...,sex,dist,trainer,driver_id,trainer_id,avisTrainer,recordAbs,gain,ecurie,incident
0,INES PICARDE,1,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,1.0,SANS_OEILLERES,Ecurie J-F. SENET,J.F. SENET,...,1,2100,https://www.letrot.com/stats/fiche-homme/jf-se...,bGV5ZwMDeQ,bGV5ZwMDeQ,2,725,48800,NaN,NaN
1,INES DE LA ROUVRE,2,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,2.0,SANS_OEILLERES,Ecurie L.M. DAVID,L.M. DAVID,...,1,2100,https://www.letrot.com/stats/fiche-homme/lm-da...,ZmRaYwYFdA,ZmRaYwYFdA,1,731,48680,NaN,NaN
2,ICE QUICK,3,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,3.0,SANS_OEILLERES,Ecurie QUICK STAR,P.J. CORDEAU,...,1,2100,https://www.letrot.com/stats/fiche-homme/pj-co...,Y2J6YwUDdw,ZmF6bQUAeQ,1,729,51310,NaN,NaN
3,IVANA DE BERTRANGE,4,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,4.0,SANS_OEILLERES,Patrice GENTIL,B. MARIE,...,1,2100,https://www.letrot.com/stats/fiche-homme/b-mar...,YmFabAUFYA,YmFabAUFYA,2,730,54310,NaN,NaN
4,INFINITY JET,5,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,5.0,SANS_OEILLERES,Mme Sylvie SAINT GEORGES,L. DURANTET,...,1,2100,https://www.letrot.com/stats/fiche-homme/l-dur...,ZmB7YQECZg,ZmB7YQECZg,2,730,52030,NaN,NaN
5,IDYLLE DE MAHEY,6,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,6.0,SANS_OEILLERES,M. SOULAS,CH. MOTTIER,...,1,2100,https://www.letrot.com/stats/fiche-homme/c-mot...,Z2Z6ZwECfA,YGJaYwUFaA,2,747,49050,NaN,NaN
6,ISSIA DE CHARLY,7,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,7.0,SANS_OEILLERES,TH. BLOQUET,P. GODEY,...,1,2100,https://www.letrot.com/stats/fiche-homme/p-god...,YmZabQUFfw,YGNbZAIAeQ,3,728,48230,NaN,NaN
7,INAYA MATIDY,8,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,8.0,SANS_OEILLERES,Cédric POSTOLLEC,T. LE BELLER,...,1,2100,https://www.letrot.com/stats/fiche-homme/t-le-...,YmJaZQoDcA,YWd4YgMGZA,1,732,50260,NaN,NaN
8,IRINUSHKA,9,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,9.0,SANS_OEILLERES,Mme M. STIHL,F. SENET,...,1,2100,https://www.letrot.com/stats/fiche-homme/f-sen...,bGV5ZwMBfw,bGV5ZwMBfw,2,734,53430,NaN,NaN
9,IRMA DU VIVIER,10,4,FEMELLES,TROTTEUR FRANCAIS,PARTANT,10.0,SANS_OEILLERES,Ecurie MINIER,B. ROBIN,...,1,2100,https://www.letrot.com/stats/fiche-homme/b-rob...,YGJaZwQFZA,YGJaZwQFZA,2,733,47970,NaN,NaN


In [536]:
for i in df.dtypes.items():
    t = str(i[1]).replace("object", "TEXT").replace("int64", "INT").replace("float64", "FLOAT").replace("bool", "BOOLEAN")
    print(i[0],"=", f"Column({t})")

nom = Column(TEXT)
numPmu = Column(INT)
age = Column(INT)
sexe = Column(TEXT)
race = Column(TEXT)
statut = Column(TEXT)
placeCorde = Column(FLOAT)
oeilleres = Column(TEXT)
proprietaire = Column(TEXT)
entraineur = Column(TEXT)
driver = Column(TEXT)
driverChange = Column(BOOLEAN)
indicateurInedit = Column(BOOLEAN)
musique = Column(TEXT)
nombreCourses = Column(INT)
nombreVictoires = Column(INT)
nombrePlaces = Column(INT)
nomPere = Column(TEXT)
nomMere = Column(TEXT)
ordreArrivee = Column(FLOAT)
jumentPleine = Column(BOOLEAN)
engagement = Column(BOOLEAN)
supplement = Column(INT)
handicapDistance = Column(INT)
poidsConditionMonteChange = Column(BOOLEAN)
tempsObtenu = Column(FLOAT)
reductionKilometrique = Column(FLOAT)
allure = Column(TEXT)
robe_code = Column(TEXT)
robe_libelleCourt = Column(TEXT)
robe_libelleLong = Column(TEXT)
dernierRapportDirect_typePari = Column(TEXT)
dernierRapportDirect_rapport = Column(FLOAT)
dernierRapportDirect_typeRapport = Column(TEXT)
dernierRapportDirect_indica

_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=TypeError("'NoneType' object is not iterable")>
Traceback (most recent call last):
  File "C:\Users\raves\anaconda3\lib\asyncio\tasks.py", line 280, in __step
    result = coro.send(None)
  File "<ipython-input-504-2bfb74fc93d6>", line 152, in _info_tableau_partant
    chevaux.extend(tableau_pmu)
TypeError: 'NoneType' object is not iterable
